In [1]:
# import pandas as pd
import csv
from pathlib import Path
import sys
from subprocess import call

# from xlsx2csv import Xlsx2csv

In [60]:
# write an excel to csv script to file ExcelToCsv.vbs
# https://stackoverflow.com/questions/28766133/faster-way-to-read-excel-files-to-pandas-dataframe
# vbscript="""if WScript.Arguments.Count < 3 Then
#     WScript.Echo "Please specify the source and the destination files. Usage: ExcelToCsv <xls/xlsx source file> <csv destination file> <worksheet number (starts at 1)>"
#     Wscript.Quit
# End If

# csv_format = 6

# Set objFSO = CreateObject("Scripting.FileSystemObject")

# src_file = objFSO.GetAbsolutePathName(Wscript.Arguments.Item(0))
# dest_file = objFSO.GetAbsolutePathName(WScript.Arguments.Item(1))
# worksheet_number = CInt(WScript.Arguments.Item(2))

# Dim oExcel
# Set oExcel = CreateObject("Excel.Application")

# Dim oBook
# Set oBook = oExcel.Workbooks.Open(src_file)
# oBook.Worksheets(worksheet_number).Activate

# oBook.SaveAs dest_file, csv_format

# oBook.Close False
# oExcel.Quit
# """;
# f = open('ExcelToCsv.vbs','w')
# f.write(vbscript)
# f.close()

In [3]:
excel = 'Nilerat_lipids_CD33_unaligned.xlsx'
csv = 'test.csv' 
call(['cscript.exe', 'ExcelToCsv.vbs', excel, csv, '1'])

0

In [56]:
path = Path('..\\..\\data\\search\\CD33\\')
testfilename = Path('excel_test_file.xlsx')
xlsxfilename = Path('Nilerat_lipids_CD33_unaligned.xlsx')
outfile = str(path / (xlsxfilename.stem + '.csv'))
# Xlsx2csv(str(path / xlsxfilename), 
#          skip_trailing_columns=False,
#          outputencoding='utf-8').convert(outfile)

In [57]:
from openpyxl import load_workbook
wb = load_workbook(filename=path / xlsxfilename, read_only=True)
ws = wb[wb.sheetnames[0]]

for row in ws.rows:
#     print(type(row))
    pass
#     print(row[0].value)
#     print(row[0].value)
    for cell in row:
#         print(cell.value)
        pass

# Close the workbook after reading
wb.close()

In [3]:
def get_headers1(line):
    d = {}
    for h in ['Checked', 'Name', 'Calc. MW','RT [min]', 'Area (Max.)', 'MS2'] :
        d[h] = line.index(h)
    for h in line:
        if 'Area: ' in h:
            d[h] = line.index(h)
    return d

def get_headers2(line):
    d = {}
    for h in ['Checked', 'Calc. MW', 'RT [min]', 'FWHM [min]', 'Max. # MI', '# Adducts', 
              'Area (All Ions)', 'Study File ID']:
        d[h] = line.index(h)
    return d

def get_headers3(line):
    d = {}
    for h in ['Checked', 'Ion', 'Charge', 'Molecular Weight', 'm/z', 'RT [min]', 'FWHM [min]', 
              '# MI', 'Area', 'Parent Area [%]', 'Study File ID']:
        d[h] = line.index(h)
    return d

def get_unaligned_headers(line):
    d = {}
    for h in ['Checked', 'Ion', 'Charge', 'Molecular Weight', 'm/z', 'RT [min]',
              'FWHM [min]', '# MI', 'Area', 'Parent Area [%]', 'Study File ID'] :
        d[h] = line.index(h)
    return d  

Checked	Name	Molecular Weight	RT [min]	Area (Max.)	MS2

Checked	Name	Molecular Weight	RT [min]	Area (Max.)	MS2

Checked	Molecular Weight	RT [min]	FWHM [min]	Max. # MI	# Adducts	Area (All Ions)	Study File ID

Checked	Molecular Weight	RT [min]	FWHM [min]	Max. # MI	# Adducts	Area	Study File ID

Checked	Ion	Charge	Molecular Weight	m/z	RT [min]	FWHM [min]	# MI	Area	Parent Area [%]	Study File ID

Checked	Ion	Charge	Molecular Weight	m/z	RT [min]	FWHM [min]	# MI	Area	Parent Area [%]	Study File ID


In [103]:
import sys
import csv
from pathlib import Path
import argparse
from subprocess import call

def get_headers1(line):
    d = {}
    for h in ['Checked', 'Name', 'Calc. MW','RT [min]', 'Area (Max.)', 'MS2'] :
        d[h] = line.index(h)
    for h in line:
        if 'Area: ' in h:
            d[h] = line.index(h)
    return d

def get_headers2(line):
    d = {}
    for h in ['Checked', 'Calc. MW', 'RT [min]', 'FWHM [min]', 'Max. # MI', '# Adducts', 
              'Area (All Ions)', 'Study File ID']:
        d[h] = line.index(h)
    return d

def get_headers3(line):
    d = {}
    for h in ['Checked', 'Ion', 'Charge', 'Molecular Weight', 'm/z', 'RT [min]', 'FWHM [min]', 
              '# MI', 'Area', 'Parent Area [%]', 'Study File ID']:
        d[h] = line.index(h)
    return d

def get_unaligned_headers(line):
    d = {}
    for h in ['Checked', 'Ion', 'Charge', 'Molecular Weight', 'm/z', 'RT [min]',
              'FWHM [min]', '# MI', 'Area', 'Parent Area [%]', 'Study File ID'] :
        d[h] = line.index(h)
    return d  

def convert_cd33(filepath, is_aligned: bool, outfilename: str=None):
    filepath = Path(filepath)
    if filepath.suffix == '.xlsx':
        print('Excel file detected. Converting to .csv')
        csv_outfile = filepath.parent / (filepath.stem + '.csv')
        call(['cscript.exe', 'ExcelToCsv.vbs', filepath, csv_outfile, '1'])
        filepath = csv_outfile
    if outfilename is None:
        outfilepath = filepath.parent / (filepath.stem + '_CD33converted' + filepath.suffix)
    else:
#         if '.csv' not in outfilename:
#             outfilename += '.csv'
        outfilepath = filepath.parent / outfilename
    if is_aligned: 
        with open(str(filepath), encoding='utf-8') as inf:
            with open(str(outfilepath), 'w', encoding='utf-8', newline='') as out:
                r = csv.reader(inf)
                w = csv.writer(out)
                # Get indexes of headers
                first_line = next(r)
                h1 = get_headers1(first_line) 
                first_header = first_line[0].strip()  # e.g. "Tags" or "Checked" or whatever CD puts in 1st column
                if type(first_header) == bytes:
                    # Handle the utf-8 BOM (byte order mark) that arises in some CSVs
                    first_header = first_header.decode(errors='ignore')
                next(r)
                second_header_line = next(r)
                h2 = get_headers2(second_header_line)
                second_header = second_header_line[1]
                next(r)
                third_header_line = next(r)
                h3 = get_headers3(third_header_line)
                third_header = third_header_line[2]
#                 for line in r:
# #                     print(type(line[1]), line[1], line[2], '\n')
#                     if line[1] == first_header:
#                         print('reached h2')
#                         h2 = get_headers2(line)
#                     elif line[2] == first_header:
#                         print('reached h3')
#                         h3 = get_headers3(line)
#                         break
                print('first_header:', first_header, '\nsecond_header:', second_header, '\nthird_header:', third_header)
                if 'RT [min]' not in h2 or 'Study File ID' not in h3:
                    print('header2:', h2, '\nheader3:', h3)
                    raise ValueError('Headers not found. Do you have multiple compounds in the first 6 rows?')
                mod_h2 = list(h2.keys())
                for i, h in enumerate(mod_h2):
                    if h == 'Calc. MW':
                        mod_h2[i] = 'Molecular Weight'
                    if h == 'Area (All Ions)':
                        mod_h2[i] = 'Area'
                inf.seek(0)  # Reset reader position to top of file
                r = csv.reader(inf)  # Reset the csv reader
                prev_line_type = None  # 'h1', 'd1', 'h2', 'd2', 'h3', 'd3'
                for i, line in enumerate(r):
                    
                    # Feature data
                    if line[1] == '' and line[2] != third_header:
                        w.writerow(['', ''] + [line[i] for i in h3.values()] + [''])
                    # Feature header
                    elif line[1] == '' and line[2] == third_header:
                        w.writerow(['', ''] + list(h3.keys()) + [''])
                    # Compound per File data:
                    elif line[0] == '' and line[1] != second_header:
                        w.writerow([''] + [line[i] for i in h2.values()] + [''])
                    # Compound per File header:
                    elif line[0] == '' and line[1] == second_header:
                        w.writerow([''] + mod_h2 + [''])
                    # Compound data
                    elif line[0] != first_header:
                        row = [line[i] for i in h1.values()] + ['']
                        row[list(h1.keys()).index('Name')] = ''  # Name field = Empty String because LipiDex parser doesn't like it
                        w.writerow(row)
                    # Compound header
                    elif line[0] == first_header:
                        w.writerow([('Molecular Weight' if 'Calc. MW' in key else key) for key in h1.keys()] + [''])
                    else:
                        raise ValueError('Could not parse line ' + str(i + 1) + ' (1-indexed)\n' + str(line))
#                     # Compound header
#                     if first_header in line[0]:
#                         w.writerow([('Molecular Weight' if 'Calc. MW' in key else key) for key in h1.keys()] + [''])
#                         prev_line_type = 'h1'
#                     # Compound data
#                     elif line[0] == '' and line[1] != first_header:
#                         row = [line[i] for i in h1.values()] + ['']
#                         row[list(h1.keys()).index('Name')] = ''  # Name field = Empty String because LipiDex parser doesn't like it
#                         w.writerow(row)
#                         prev_line_type = 'd1'
#                     # Compound per File header:
#                     elif line[0] == '' and line[1] == first_header:
#                         w.writerow([''] + mod_h2 + [''])
#                     # Compound per File data:
#                     elif line[1] == '' and line[2] != first_header:
#                         w.writerow([''] + [line[i] for i in h2.values()] + [''])
#                     # Feature header:
#                     elif line[1] == '' and line[2] == first_header:
#                         w.writerow(['', ''] + list(h3.keys()) + [''])
#                     # Feature data:
#                     elif line[2] == '':
#                         w.writerow(['', ''] + [line[i] for i in h3.values()] + [''])
                    
    else: 
        with open(str(filepath), encoding='utf8') as inf:
            with open(str(outfilepath), 'w', encoding='utf8', newline='') as out:
                r = csv.reader(inf)
                w = csv.writer(out)
                # Get indexes of headers
                first_line = next(r)
                header = get_unaligned_headers(first_line) 
                w.writerow(list(header.keys()) + [''])
                for line in r:
                    w.writerow([line[i] for i in header.values()] + [''])

In [102]:
aligned = r'../../data/search/CD33/Nilerat_lipids_CD33_aligned.csv'
unaligned = r'../../data/search/CD33/Nilerat_lipids_CD33_unaligned.csv'

convert_cd33(aligned, True)
convert_cd33(unaligned, False)

first_header: ﻿Tags 
second_header: Tags 
third_header: Tags


## Unaligned file CD 3.3 converter

Checked	Ion	Charge	Molecular Weight	m/z	RT [min]	FWHM [min]	# MI	Area	Parent Area [%]	Study File ID


Checked	Ion	Charge	Molecular Weight	m/z	RT [min]	FWHM [min]	# MI	Area	Parent Area [%]	Study File ID


In [69]:
def get_unaligned_headers(line):
    d = {}
    for h in ['Checked', 'Ion', 'Charge', 'Molecular Weight', 'm/z', 'RT [min]',
              'FWHM [min]', '# MI', 'Area', 'Parent Area [%]', 'Study File ID'] :
        d[h] = line.index(h)
    return d  

with open(r'../data/search/CD33/Nilerat_lipids_CD33_unaligned.csv', encoding='utf8') as inf:
    with open(r'..\data\search\CD33\unaligned_test.csv', 'w', encoding='utf8', newline='') as out:
        r = csv.reader(inf)
        w = csv.writer(out)
        # Get indexes of headers
        first_line = next(r)
        header = get_unaligned_headers(first_line) 
        w.writerow(list(header.keys()))
        for line in r:
            w.writerow([line[i] for i in header.values()])

In [143]:
parser = argparse.ArgumentParser()
parser.add_argument('-f', '--file', dest='filepath', type=str, 
                    help='CD 3.3 .csv file to convert')
aligned_flag = parser.add_mutually_exclusive_group()
aligned_flag.add_argument('-a', '--aligned', dest='is_aligned', action='store_true')
aligned_flag.add_argument('-u', '--unaligned', dest='is_aligned', action='store_false')
parser.add_argument('-o', '--outfilename', dest='outfilename', default=None,
                    help='(Optional) Filename for converted file')
    

_StoreAction(option_strings=['-o', '--outfilename'], dest='outfilename', nargs=None, const=None, default=None, type=None, choices=None, help='(Optional) Filename for converted file', metavar=None)

In [149]:
args = parser.parse_args(['-f', 'asdf', '--aligned',])
print(args.outfilename)

None


In [142]:
parser.parse_args().filepath

'C:\\Users\\bjanderson23\\AppData\\Roaming\\jupyter\\runtime\\kernel-d53dbdbd-323b-49a9-bb24-030dd55a26f3.json'